In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import logging
import torch
from torchsummary import summary


logging.basicConfig(level=logging.INFO)

from src.data_loader.data_loader import DataLoader
from src.model.model import TwoHeadConvNeXtV2
from src.config.configuration import CLASS_NUM

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
logging.info(f"Using device: {device}")
! nvidia-smi

In [ ]:
img_path = "data/train_images_small"
label_path = "data/venomous_status_metadata.csv"
meta_data_path = "data/train_images_metadata.csv"

data_loader = DataLoader(
    image_data_set_path=img_path,
    meta_data_path=meta_data_path,
    label_info_path=label_path
    )

In [ ]:
model = TwoHeadConvNeXtV2(num_multi_classes=CLASS_NUM)

In [ ]:
summary(model, input_size=(3, 224, 224), device=str(device))

In [ ]:
from src.model.utils import train_model
from src.data_loader.augmentation import Augmentor
augmentor = Augmentor(num_augmentations=40000, center_n_transforms=2, center_magnitude=10)
train_model(data_loader, model, augmentor=augmentor)